# Analysis for Metrics 1- 5

## Lily Cao

Using Python, I was able to create dataframes and csvs that I later used to map my calculations. The five key metrics I calculated here are:  

1.	The rate of crimes committed during an outage with respect to crimes committed within a buffer [City level]
2.	The rate of crimes committed during an outage with respect to crimes committed within that same buffer [Buffer level]
3.	The rate of crimes committed during an outage with respect to all crimes committed in Chicago in 2016 [City level]
4.	The rate of crimes committed during an outage with respect to the outage time period [Buffer level]
5.	The rate of crimes committed pre-/post-outage with respect to the non-outage time period [Buffer level]

After importing the csvs downloaded from the Chicago Data Portal, I created dataframes for each and dropped insignificant columns. With the streetlight outage csvs, I merged them and dropped any duplicate requests.

In [2]:
import pandas as pd
street_all_out = pd.read_csv('Street All Out.csv')
street_one_out = pd.read_csv('Street One Out.csv')
crimes = pd.read_csv("Crimes.csv")
df = pd.concat([street_all_out, street_one_out])
df = df[~df.Status.str.contains('Dup')]
df = df.dropna()
crimes = crimes.drop(columns=['Case Number', 'Block', 'Beat', 'IUCR', 'Description', 'Community Area', \
                             'FBI Code', 'Updated On', 'Year', 'Domestic'])
crimes = crimes.dropna()

I narrow down my analysis to crimes and outages that occurred during 2016. While the homicide rate remained steady between 2010 and 2015, it increased dramatically in 2016, becoming the highest homicide rate in Chicago since the mid-1990s.


In [3]:
out_2016 = df.loc[df['Creation Date'].str.contains('2016')]
crime_2016= crimes.loc[crimes['Date'].str.contains('2016')]
out_2016.iloc[:,7:9].head()

,X Coordinate,Y Coordinate
152147,1.183375e+06,1.848798e+06
152148,1.152592e+06,1.893323e+06
152149,1.171092e+06,1.843569e+06
152150,1.183402e+06,1.847917e+06
152151,1.183064e+06,1.848128e+06


Dataframe 'df' includes all Euclidean distances between each crime and outage. Columns represent indices of crime_2016 (aka the crimes). Rows represent indices of out_2016 (aka the outages).

In [4]:
import scipy
import scipy.spatial
ary = scipy.spatial.distance.cdist(out_2016.iloc[:,7:9], crime_2016.iloc[:,7:9], metric='euclidean')
df = pd.DataFrame(ary)
df.head() 

,0,1,2,3,4,5,6,7,8,9,...,42728,42729,42730,42731,42732,42733,42734,42735,42736,42737
0,48188.511359,15714.464329,34567.768129,56580.236817,51696.399048,77953.709420,68785.386755,57099.913885,31899.733426,50044.994841,...,60522.093449,45968.012375,30545.611272,19179.123040,52047.581844,21400.956700,12057.613877,14703.183479,8821.805073,13174.122732
1,20727.830974,62846.254927,26759.924994,3597.691240,16723.984653,24150.904368,14768.723497,8279.614986,22233.315514,4120.246025,...,18953.904005,13856.420389,31650.223752,36440.763237,36919.019839,66168.365943,43903.885276,62037.878727,47340.472233,56360.430669
2,52362.877059,9863.723871,28828.235110,56094.137636,54812.943487,77212.243906,66980.768236,57811.139724,32102.332149,49268.299944,...,63399.063059,48154.441640,37011.018986,17027.029847,42291.682943,13389.242533,10796.285822,27826.998158,8971.643686,26448.236830
3,49055.428455,14958.446497,35052.191340,57343.910179,52548.591754,78722.759523,69512.478596,57900.357736,32646.608160,50792.027401,...,61373.983450,46797.128388,31425.932489,19742.039252,52304.234498,20657.145930,12504.466540,14832.115778,9167.901590,13635.847065
4,48778.603756,14977.396368,34652.720469,56986.779545,52250.541311,78367.590864,69142.383439,57560.856510,32283.827312,50428.716484,...,61075.398121,46477.099580,31190.089585,19345.555675,51921.037986,20665.546906,12105.138590,15122.766880,8772.555826,13797.301286


Outages is a list of dictionaries. Each non-empty dictionary maps a crime to its distance from an outage; all distances are less than or equal to 164.043 ft (50 meters)

For instance, the dictionary at index 1 of the list contains 6 crimes. This tells us that the outage at index 1 of out_2016 has 6 crimes (at indices 3744, 16408, 17777, 19327, 21697, and 26915 of crime_2016) within 50 meters.

In [5]:
outages = []
for i in range(df.shape[0]):
    outages.append(df.iloc[i][df.iloc[i] <= 164.042].to_dict())

I created a dataframe named during_outage. I first added two columns: 'outages' and 'crimes' by using the outages list I just created above. The 'outages' column contains indeces of outages from out_2016 that have crimes within 50 meters of it. The indeces of these crimes from crime_2016 make up the 'crimes' column here.  

Essentially, I turned each dictionary from the outages list above into a table. Then, I merged every table.  

We can use the same example from above--the dictionary for the outage at index 1 of out_2016 to see section of the merged table:

In [32]:
indices = [i for i,x in enumerate(outages) if x]
crimes_within_buffer = 0
df_list = []
for i in indices:
    df1 = pd.DataFrame.from_dict(outages[i].keys())
    df1.insert(0, 'outages', [i]*len(outages[i]))
    df1.rename(columns={0:'crimes'}, inplace=True)
    crimes_within_buffer += len(df1)
    df_list.append(df1)
    
during_outage = pd.concat(df_list)
during_outage.loc[during_outage['outages'] == 1]

,outages,crimes
0,1,3744
1,1,16408
2,1,17777
3,1,19327
4,1,21697
5,1,26915


To avoid duplicating data, I added a column titled 'street' whose cells represent the street addresses of each street light that had at least one outage. From now on, instead of analyzing the data with respect to outages, I will do so with respect to streetlight addresses.  

I added the following columns to the during_outage dataframe:  
**crime time:** the date and time at which the crime was reported  
**outage creation date:** the date on which the outage was reported  
**outage completion date:** the date on which the outage was fixed  
**during out:** a Boolean that tells us whether the crime was committed during the outage period

In [33]:
crime_time = []
for i in during_outage['crimes']:
    crime_time.append(crime_2016.iloc[i]['Date'])

outage_creation = []
outage_complete = []
for i in during_outage['outages']:
    outage_creation.append(out_2016.iloc[i]['Creation Date'])
    outage_complete.append(out_2016.iloc[i]['Completion Date'])

streets = []
for i in during_outage['outages']:
    streets.append(out_2016.iloc[i]['Street Address'])
    
during_outage['crime time'] = crime_time
during_outage['outage creation date'] = outage_creation
during_outage['outage completion date'] = outage_complete
during_outage.insert(loc=0, column='street', value = streets)
during_outage = during_outage.sort_values(by='street')

In [34]:
during_outage['during out'] = (during_outage['crime time'] < during_outage['outage completion date']) & \
              (during_outage['crime time'] >= during_outage['outage creation date'])

Now, I can get 1) the number of crimes committed within a buffer and during an outage:

In [35]:
during_outage['during out'].value_counts()

False    16758
True       262
Name: during out, dtype: int64

and 2) the number of crimes committed within a buffer:

In [36]:
crimes_within_buffer

17020

With these two raw counts, I can calculate the <span style="color:red"> **first metric** </span>:

262/17020 = **1.539%**

Next, to get the second metric, I have to calculate a similar rate but for **each** street light. I can get each rate by dividing the number of crimes committed during an outage and within a buffer by the total number of crimes committed within that same buffer.

I'll be calculating this rate for 6,226 (unique) street lights

In [37]:
during_outage['street'].nunique()

6226

In [38]:
all_lights = during_outage['street'].unique()

rates = []
num_crime_buffer = []
num_crime_during = []

for light in all_lights:
    d = during_outage.loc[during_outage['street'] == light]
    num_crime_buffer.append(d.shape[0])
    
    d2 = d[(d['crime time'] < d['outage completion date']) & \
           (d['crime time'] >= d['outage creation date'])]
    num_crime_during.append(d2.shape[0])
    
    rate = len(d2)/len(d)
    rates.append(rate)
    
each_street = pd.DataFrame(list(zip(all_lights, num_crime_during, num_crime_buffer, rates)), \
                           columns = ['street', 'crimes during out', 'crimes in buffer','rate'])
each_street.head()

,street,crimes during out,crimes in buffer,rate
0,1 E 118TH ST,0,2,0.0
1,1 E 63RD ST,0,12,0.0
2,1 E 69TH ST,2,2,1.0
3,1 E WACKER DR,0,1,0.0
4,1 N CENTRAL AVE,0,12,0.0


Thus, the <span style="color:red"> **second metric** </span> can be found in the 'rate' column of the each_street dataframe above.

To map these rates/ratios in QGIS, I add columns for the longitudes and latitudes of each street light:

In [39]:
long = []
lat = []
for light in all_lights:
    i = out_2016.loc[out_2016['Street Address'] == light].index[0]    
    long.append(out_2016.loc[i]['Longitude'])
    lat.append(out_2016.loc[i]['Latitude'])

each_street['long'] = long
each_street['lat'] = lat

each_street.head()

,street,crimes during out,crimes in buffer,rate,long,lat
0,1 E 118TH ST,0,2,0.0,-87.622716,41.679850
1,1 E 63RD ST,0,12,0.0,-87.625375,41.780058
2,1 E 69TH ST,2,2,1.0,-87.624942,41.769194
3,1 E WACKER DR,0,1,0.0,-87.627975,41.886814
4,1 N CENTRAL AVE,0,12,0.0,-87.764864,41.880246


For the <span style="color:red"> **third metric** </span>, I need to calculate the number of crimes committed during an outage and within a buffer; and the total number of crimes committed (in Chicago in 2016). 

262 crimes were committed during an outage and within a buffer. The total number of crimes commtited is 42738. These two numbers give us a very low rate of 262/42738 = 0.613%  

There were 236 street lights (out of 6226) with an individual crime within buffer rate greater than the Chicago average. That’s 3.791% percent of all street lights--a small percent. 

In [81]:
during_outage['during out'].value_counts()

False    16758
True       262
Name: during out, dtype: int64

In [82]:
len(crime_2016)

42738

In [83]:
each_street['above Chicago avg.'] = each_street['rate'] > 0.00613037577
each_street.head()

,street,crimes during out,crimes in buffer,rate,long,lat,above Chicago avg.
0,1 E 118TH ST,0,2,0.0,-87.622716,41.679850,False
1,1 E 63RD ST,0,12,0.0,-87.625375,41.780058,False
2,1 E 69TH ST,2,2,1.0,-87.624942,41.769194,True
3,1 E WACKER DR,0,1,0.0,-87.627975,41.886814,False
4,1 N CENTRAL AVE,0,12,0.0,-87.764864,41.880246,False


In [84]:
len(each_street[each_street['above Chicago avg.'] == True])/len(each_street) 

0.037905557340186316

For the <span style="color:red"> **fourth metric** </span>, I need to calculate the number of crimes committed during an outage for each buffer and the outage time period.

By subtracting the outage creation date from the completion date, I get the number of *days* for which the street light was out. To avoid division by zero errors, I add one day to the time difference. Since I'm doing this after, I'm not (incorrectly) affecting my calculation of the number of crimes committed during an outage.

I add another column for calculations of the third metric. Each cell in this column is the quotient of one divided by time difference. I chose one as the divisor because each row represents *one* crime.

In [85]:
during_outage1 = during_outage.copy()
during_outage1['crime time'] = pd.to_datetime(during_outage1['crime time'])
during_outage1['outage creation date'] = pd.to_datetime(during_outage1['outage creation date'])
during_outage1['outage completion date'] = pd.to_datetime(during_outage1['outage completion date']) 

during_outage1['time diff'] = during_outage1['outage completion date'] - \
                            during_outage1['outage creation date'] 
during_outage1['time diff'] = during_outage1['time diff'].dt.days + 1

during_outage1['4th metric'] = 1 / during_outage1['time diff']

fourth_m = during_outage1.loc[during_outage1['during out'] == True]
fourth_m.head()

,street,outages,crimes,crime time,outage creation date,outage completion date,during out,time diff,4th metric
1,1 E 69TH ST,5469,20063,2016-07-06 12:45:00,2016-06-01,2016-07-30,True,60,0.016667
0,1 E 69TH ST,5469,16049,2016-06-02 11:30:00,2016-06-01,2016-07-30,True,60,0.016667
6,1 W 87TH ST,7221,21559,2016-07-18 13:40:00,2016-07-16,2016-07-20,True,5,0.200000
5,1 W HARRISON ST,11424,31989,2016-10-10 01:00:00,2016-10-06,2016-10-12,True,7,0.142857
0,10 W 17TH ST,4277,11862,2016-04-26 18:00:00,2016-04-25,2016-04-27,True,3,0.333333


By adding up the fourth metrics for each crime, I get the the correct fourth metric for each street light:

In [86]:
sum_list = []
for street in fourth_m['street'].unique():
    fourth_metrics = fourth_m.loc[fourth_m['street'] == street]['4th metric']
    sum_list.append(sum(fourth_metrics))

fourth_m = pd.DataFrame(fourth_m['street'].unique(), columns = ['street'])
fourth_m['4th metric'] = sum_list
fourth_m.head()

,street,4th metric
0,1 E 69TH ST,0.033333
1,1 W 87TH ST,0.200000
2,1 W HARRISON ST,0.142857
3,10 W 17TH ST,0.333333
4,100 E RANDOLPH ST,0.500000


For the <span style="color:red"> **fifth metric** </span>, I need to calculate the number of crimes committed pre-/post-outage for each buffer and the non-outage time period.


First, I created a dataframe mapping each street light to the number of days during which there was no outage. Then, for each street light, I counted the number of crimes that did not occur during an outage (aka crimes whose 'during out' column was False). By dividing these two lists, I could create a column called '5th metric' that gives me the desired rate.

In [88]:
during_outage2 = during_outage1.drop_duplicates('outages', inplace = False)
d_o3 = during_outage2.groupby(['street'])['time diff'].sum().reset_index()
d_o3['num days no outage'] = 365 - d_o3['time diff']
d_o3.head()

,street,time diff,num days no outage
0,1 E 118TH ST,16,349
1,1 E 63RD ST,24,341
2,1 E 69TH ST,60,305
3,1 E WACKER DR,4,361
4,1 N CENTRAL AVE,4,361


In [89]:
d_o = during_outage1
num_crimes_not_out = []
for street in all_lights:
    c = d_o.loc[(d_o['street'] == street) & (d_o['during out'] == False)]
    num_crimes_not_out.append(len(c))
    
fourth_half = pd.DataFrame(all_lights, columns = ['street'])
fourth_half['crimes not during out'] = num_crimes_not_out
fourth_half['non-outage time'] = list(d_o3['num days no outage'])
fourth_half['5th metric'] = fourth_half['crimes not during out'] / fourth_half['non-outage time']
fourth_half.head()

,street,crimes not during out,non-outage time,5th metric
0,1 E 118TH ST,2,349,0.005731
1,1 E 63RD ST,12,341,0.035191
2,1 E 69TH ST,0,305,0.000000
3,1 E WACKER DR,1,361,0.002770
4,1 N CENTRAL AVE,12,361,0.033241


I merged these two dataframes to get a complete overview of the metrics for each street light:

In [90]:
fourth_metric_merged = pd.DataFrame(all_lights, columns = ['street'])

lst = []
for street in all_lights:
    if street in list(fourth_m['street']):
        i = fourth_m[fourth_m['street'] == street].index[0]
        lst.append(fourth_m.iloc[i]['4th metric'])
    else:
        lst.append(0)

fourth_metric_merged['4th metric'] = lst 
fourth_metric_merged['5th metric'] = fourth_half['5th metric']

fourth_metric_merged.head() 

,street,4th metric,5th metric
0,1 E 118TH ST,0.000000,0.005731
1,1 E 63RD ST,0.000000,0.035191
2,1 E 69TH ST,0.033333,0.000000
3,1 E WACKER DR,0.000000,0.002770
4,1 N CENTRAL AVE,0.000000,0.033241


To compare these rates for each street light, I made a new boolean column that tells us whether the 4th metric is greater than the 5th metric. The 4th metric tells us the rate of crimes being committed during an outage while the 5th metric tells us the rate of crimes being committed pre-/post-outage. If the former is greater than the latter, then there may be a positive relationship between street light outage and crime (rate).

In [91]:
fourth_metric_merged['4th > 5th metric'] = (fourth_metric_merged['4th metric'] > fourth_metric_merged['5th metric'])
fourth_metric_merged.head()

,street,4th metric,5th metric,4th > 5th metric
0,1 E 118TH ST,0.000000,0.005731,False
1,1 E 63RD ST,0.000000,0.035191,False
2,1 E 69TH ST,0.033333,0.000000,True
3,1 E WACKER DR,0.000000,0.002770,False
4,1 N CENTRAL AVE,0.000000,0.033241,False


To map these rates/ratios in QGIS, I add columns for the longitudes and latitudes of each street light:

In [92]:
long = []
lat = []
for light in all_lights:
    i = out_2016.loc[out_2016['Street Address'] == light].index[0]    
    long.append(out_2016.loc[i]['Longitude'])
    lat.append(out_2016.loc[i]['Latitude'])

fourth_metric_merged['longitude'] = long
fourth_metric_merged['latitude'] = lat

fourth_metric_merged.head()

,street,4th metric,5th metric,4th > 5th metric,longitude,latitude
0,1 E 118TH ST,0.000000,0.005731,False,-87.622716,41.679850
1,1 E 63RD ST,0.000000,0.035191,False,-87.625375,41.780058
2,1 E 69TH ST,0.033333,0.000000,True,-87.624942,41.769194
3,1 E WACKER DR,0.000000,0.002770,False,-87.627975,41.886814
4,1 N CENTRAL AVE,0.000000,0.033241,False,-87.764864,41.880246


In [93]:
len(fourth_metric_merged[fourth_metric_merged['4th > 5th metric'] == True])

231

231 street lights had higher crimes rates during an outage than during a non-outage period. That's 231/6226 = 3.710%.

In [105]:
fourth_metric_merged['metric diff'] = fourth_metric_merged['4th metric'] - \
fourth_metric_merged['5th metric']
fourth_metric_merged.head()

,street,4th metric,5th metric,4th > 5th metric,longitude,latitude,metric diff
0,1 E 118TH ST,0.000000,0.005731,False,-87.622716,41.679850,-0.005731
1,1 E 63RD ST,0.000000,0.035191,False,-87.625375,41.780058,-0.035191
2,1 E 69TH ST,0.033333,0.000000,True,-87.624942,41.769194,0.033333
3,1 E WACKER DR,0.000000,0.002770,False,-87.627975,41.886814,-0.002770
4,1 N CENTRAL AVE,0.000000,0.033241,False,-87.764864,41.880246,-0.033241


In [99]:
pd.DataFrame.mean(fourth_metric_merged)

4th metric           0.005455
5th metric           0.007564
4th > 5th metric     0.037102
longitude          -87.671581
latitude            41.830190
metric diff         -0.002109
dtype: float64

On average, the crime rate during non-outage periods is 0.00211 greater than that during outage periods.  

What's interesting, however, is that lights with higher crime rates during outage periods have a high average metric difference (0.132657)--much higher than that for lights with lower crime rates during outage periods (0.007302). 

In other words, when the crime rate during outages is higher than the crime rate during non-outage periods, it's *much* higher. On the other hand, when the crime rate during outages is lower than the crime rate during non-outage periods, it's *not that much* lower, on average.

In [100]:
pd.DataFrame.mean(fourth_metric_merged[fourth_metric_merged['4th > 5th metric'] == True])

4th metric           0.145864
5th metric           0.013208
4th > 5th metric     1.000000
longitude          -87.668447
latitude            41.824516
metric diff          0.132657
dtype: float64

In [101]:
pd.DataFrame.mean(fourth_metric_merged[fourth_metric_merged['4th > 5th metric'] == False])

4th metric           0.000045
5th metric           0.007347
4th > 5th metric     0.000000
longitude          -87.671701
latitude            41.830409
metric diff         -0.007302
dtype: float64